<a href="https://colab.research.google.com/github/suinkangme/COMP432-GroupI/blob/main/hyperparameter_search_Orion_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --fuzzy 'https://drive.google.com/file/d/1M0uN1jbjbOh3b6fjDfe2qrNnYSJrgfwY/view?usp=sharing'

Downloading...
From: https://drive.google.com/uc?id=1M0uN1jbjbOh3b6fjDfe2qrNnYSJrgfwY
To: /content/Dataset_1.zip
100% 831M/831M [00:15<00:00, 53.8MB/s]


In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

In [ ]:
!unzip /content/Dataset_1.zip

Streaming output truncated to the last 5000 lines.
  inflating: Colorectal Cancer/MUS/MUS-CIHSSWPV.tif  
  inflating: Colorectal Cancer/MUS/MUS-CIHWTRHA.tif  
  inflating: Colorectal Cancer/MUS/MUS-CIILSKST.tif  
  inflating: Colorectal Cancer/MUS/MUS-CIIMVVTT.tif  
  inflating: Colorectal Cancer/MUS/MUS-CIITQWPP.tif  
  inflating: Colorectal Cancer/MUS/MUS-CIKANVTM.tif  
  inflating: Colorectal Cancer/MUS/MUS-CIKKELKA.tif  
  inflating: Colorectal Cancer/MUS/MUS-CILPKGCQ.tif  
  inflating: Colorectal Cancer/MUS/MUS-CIMWFGFH.tif  
  inflating: Colorectal Cancer/MUS/MUS-CINGCMFK.tif  
  inflating: Colorectal Cancer/MUS/MUS-CINGLGES.tif  
  inflating: Colorectal Cancer/MUS/MUS-CIPMCCVS.tif  
  inflating: Colorectal Cancer/MUS/MUS-CIPNEIRF.tif  
  inflating: Colorectal Cancer/MUS/MUS-CIQAGCRV.tif  
  inflating: Colorectal Cancer/MUS/MUS-CIQFFGDI.tif  
  inflating: Colorectal Cancer/MUS/MUS-CIRAICLW.tif  
  inflating: Colorectal Cancer/MUS/MUS-CISHTCWQ.tif  
  inflating: Colorectal Cancer/

In [ ]:
%%capture
!pip install git+https://github.com/epistimio/orion.git@develop
!pip install orion[profet]

In [ ]:
%%file train.py

import argparse
import numpy as np
import sklearn
import sklearn.preprocessing
import sklearn.neural_network
from orion.client import report_objective # Orion
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import seaborn as sns
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

# Apply transforms to subsets using a custom Dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

def train():
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batchsize', type=int, default=64,
                        help='input batch size for training (default: 64)')
    parser.add_argument('--epochs', type=int, default=10,
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=0.01,
                        help='learning rate (default: 1.0)')
    parser.add_argument('--eval', type=bool, default=False,
                        help='If True it prints the test error (default: False)')
    parser.add_argument("-f", required=False)
    args = parser.parse_args()

    classes = ['MUS', 'NORM', 'STR']

    dataset_dir = '/content/Colorectal Cancer'
    transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    full_dataset = datasets.ImageFolder(root=dataset_dir)
    train_indices, test_indices = train_test_split(list(range(len(full_dataset))), test_size=0.2)
    train_indices, val_indices = train_test_split(train_indices, test_size=0.1)  # 10% of 80% = 8% of the whole
    train_subset = Subset(full_dataset, train_indices)
    val_subset = Subset(full_dataset, val_indices)
    test_subset = Subset(full_dataset, test_indices)
    original_dataset = CustomDataset(train_subset, transform=transform)
    original_loader = DataLoader(original_dataset, batch_size=args.batchsize, shuffle=False)
    train_dataset = CustomDataset(train_subset, transform=transform)
    val_dataset = CustomDataset(val_subset, transform=transform)
    test_dataset = CustomDataset(test_subset, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=args.batchsize, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batchsize, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=args.batchsize, shuffle=False)

    model = resnet18(weights = None)
    num_classes = len(classes)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)

    optimizer = optim.Adam(model.parameters(), lr = args.lr)
    criterion = nn.CrossEntropyLoss()

    # Training
    # Training the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(args.epochs):
      model.train()
      running_loss = 0.0
      correct = 0
      total = 0
      for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #running_loss += loss.item()

    # validation
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
      for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    valid_error = 100*(1 - (correct/total))
    print("Valid Error (\%): " + str(valid_error))

    report_objective(valid_error)

    if args.eval:
      running_loss = 0
      total = 0
      correct = 0
      with torch.no_grad():
        for data in test_loader:
          images, labels = data
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          loss = criterion(outputs, labels)
          running_loss += loss.item()
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
      test_error = 100*(1-(correct/total))
      print("Test Error (\%): " + str(test_error))

if __name__ == '__main__':
    train()

Writing train.py


In [ ]:
!orion hunt -n hp_adam --exp-max-trials=10 python train.py --lr~'loguniform(1e-4, 0.1)' --batchsize~'choices([16, 32, 64])' --epochs~'uniform(15, 30, discrete=True)'

2023-11-15 17:16:37,422::WARNING::root::Script /content/train.py is not in a git repository. Code modification won't be detected.
2023-11-15 17:16:37,543::WARNING::root::Script /content/train.py is not in a git repository. Code modification won't be detected.
2023-11-15 17:16:39,856::WARNING::root::Script /content/train.py is not in a git repository. Code modification won't be detected.
2023-11-15 17:16:40,417::WARNING::root::Script /content/train.py is not in a git repository. Code modification won't be detected.
Valid Error (\%): 5.41666666666667
2023-11-15 17:21:54,050::WARNING::root::Script /content/train.py is not in a git repository. Code modification won't be detected.
Valid Error (\%): 4.583333333333329
2023-11-15 17:25:22,931::WARNING::root::Script /content/train.py is not in a git repository. Code modification won't be detected.
Valid Error (\%): 6.874999999999998
2023-11-15 17:29:03,122::WARNING::root::Script /content/train.py is not in a git repository. Code modification wo

In [ ]:
!orion info --name hp_adam --version 1

Identification
name: hp_adam
version: 1
user: root


Commandline
python train.py --lr~loguniform(1e-4, 0.1) --batchsize~choices([16, 32, 64]) --epochs~uniform(15, 30, discrete=True)


Config
max trials: 10
max broken: 3
working dir: 


Algorithm
random:
    seed: None


Space
=====
/batchsize: choices([16, 32, 64])
/epochs: uniform(15, 30, discrete=True)
/lr: loguniform(0.0001, 0.1)


Meta-data
user: root
datetime: 2023-11-15 17:00:31.809287
orion version: 0.2.6.post333+gd51e6ea8
VCS:



Parent experiment
root:
parent:
adapter:


Stats
=====
completed: True
trials completed: 10
best trial:
  id: e27791032c81e414c15c61db3d708e8d
  evaluation: 2.708333333333335
  params:
    /batchsize: 64
    /epochs: 21
    /lr: 0.004083
start time: 2023-11-15 17:00:31.809287
finish time: 2023-11-15 17:58:41.878657
elapsed_time: 0:58:09.593601




In [ ]:
!python train.py --lr=0.004083 --epochs=21 --batchsize=64 --eval='True'

Valid Error (\%): 7.2916666666666625
[{'name': 'objective', 'type': 'objective', 'value': 7.2916666666666625}]
Test Error (\%): 6.166666666666664
